# Many time pad (WORK IN PROGRESS)

- Let us see what goes wrong when a stream cipher key is used more than once. Below are eleven hex-encoded ciphertexts that are the result of encrypting eleven plaintexts with a stream cipher, all with the same stream cipher key. Your goal is to decrypt
the last ciphertext, and submit the secret message within it as solution.
- Hint: XOR the ciphertexts together, and consider what happens when a space is XORed with a character in a-z, A-Z

## TODO
- Add writeup - detailed explanation of the solution for this problem

In [1]:
def longest_length(my_list):
    running_max = 0
    for ct in my_list:
        if len(ct) > running_max:
            running_max = len(ct)
    return running_max


def truncated(a, b):
    length = min(len(a), len(b))
    return a[:length], b[:length]


def truncated3(a, b, c):
    length = min(len(a), len(b), len(c))
    return a[:length], b[:length], c[:length]


def convert_to_bytes(hexstring):
    r = []
    #Get two characters at a time
    for i in range(len(hexstring) // 2):
        snippet = hexstring[2*i:( 2*i + 2)]
        x = int(snippet, 16)
        r.append(x)
    return r


def xor_bytes(x, y):
    a, b = truncated(x, y)
    r =[]
    for i, j in zip(a, b):
        r.append(i ^ j)
    return r


def within_range(x):
    return 65 <= x <= 90 or 97 <= x <= 122


def decode_bytes(x):
    r = ''
    for i in x:
        if within_range(i) or i == ord(" "):
            r+=chr(i)
        else:
            r+="_"
    return r


def decode(cipher, key):
    
    r = ''
    space = ord(" ")
    c, k = truncated(cipher, key)
    
    for i, j in zip(c, k):
        if j == 0:
            r+="_"
            continue

        x = i ^ j
        if within_range(x) or x == space:
            r+=chr(x)
        else:
            r+="_"
    return r 


def decode_raw(cipher, key):
    
    r = ''
    c, k = truncated(cipher, key)
    
    for i, j in zip(c, k):
        if j == 0:
            r+="_"
            continue

        x = i ^ j
        r+=chr(x)
    return r 

def build_key(key, c1, c2, c3):

    xm12 = xor_bytes(c1, c2)
    xm13 = xor_bytes(c1, c3)
    xm23 = xor_bytes(c2, c3)
    
    length = len(c1)
    space = ord(" ")
    
    for i in range(0, length):
        
        if key[i] != 0: 
            continue 
            
        if c1[i] == c2[i] or c1[i] == c3[i] or c2[i] == c3[i]:
            continue
        
        m12, m13, m23 = xm12[i], xm13[i], xm23[i]
            
        if within_range(m13) and within_range(m23):            
            key[i] = space ^ c3[i]
            #print("3: ", key[i], m13 ^ space ^ c1[i], m23 ^ space ^ c2[i])
            
        elif within_range(m12) and within_range(m23):
            key[i] = space ^ c2[i]
            #print("2: ", key[i], m12 ^ space ^ c1[i], m23 ^ space ^ c3[i])
            
        elif within_range(m12) and within_range(m13):
            key[i] = space ^ c1[i]
            #print("1: ", key[i], m12 ^ space ^ c2[i], m13 ^ space ^ c3[i])

In [2]:
c1hex = "315c4eeaa8b5f8aaf9174145bf43"
c2hex = "234c02ecbbfbafa3ed18510abd11"
c3hex = "32510ba9a7b2bba9b8005d43a304"

ciphertexts = [c1hex, c2hex, c3hex]
cipherlength = len(ciphertexts)

# each byte is 2 characters in hex  
key_length = longest_length(ciphertexts) // 2
keybytes = [0 for i in range(key_length)]

In [3]:
c1, c2, c3 = truncated3(c1hex, c2hex, c3hex)

c1bytes = convert_to_bytes(c1)
c2bytes = convert_to_bytes(c2)
c3bytes = convert_to_bytes(c3)

x12 = xor_bytes(c1bytes, c2bytes)
x13 = xor_bytes(c1bytes, c3bytes)
x23 = xor_bytes(c2bytes, c3bytes)

answer12 = convert_to_bytes('12104c06134e5709140f104f0252')
answer13 = convert_to_bytes('030d45430f07430341171c061c47')
answer23 = convert_to_bytes('111d09451c49140a55180c491e15')

x12decoded = decode_bytes(x12)
x13decoded = decode_bytes(x13)
x23decoded = decode_bytes(x23)

build_key(keybytes, c1bytes, c2bytes, c3bytes)

message1 = decode(c1bytes, keybytes)
message2 = decode(c2bytes, keybytes)
message3 = decode(c3bytes, keybytes)

print(x12 == answer12)
print(x13 == answer13)
print(x23 == answer23)

print(x12decoded=="__L__NW____O_R") 
print(x13decoded=="__EC__C_A____G") 
print(x23decoded=="___E_I__U__I__") 


print(keybytes==[0, 0, 110, 137, 0, 219, 216, 0, 152, 0, 0, 42, 0, 99])

print(message1 == "__ c_n _a__o_ ")
print(message2 == "__le_ w_u__ _r")
print(message3 == "__e _ic_ __i_g")


True
True
True
True
True
True
True
True
True
True


In [4]:
target='32510ba9babebbbefd001547a810e67149caee11d945cd7fc81a05e9f85aac650e9052ba6a8cd8257bf14d13e6f0a803b54fde9e77472db89d71b57bddef121336cb85ccb8f3315f4b52e301d16e9f52f904'

x1='315c4eeaa8b5f8aaf9174145bf43e1784b8fa00dc71d885a804e5ee9fa40b16349c146fb778cdf2d3a'
x2='234c02ecbbfbafa3ed18510abd11fa724fcda2018a1a8342cf064bbde548b12b07df44ba7191d9606ef4081'
x3='32510ba9a7b2bba9b8005d43a304b5714cc0bb0c8a34884dd91304b8ad40b62b07df44ba6e9d8a2368e51d04e0e7b207b70b9b8261112bacb6c866a232dfe257527dc29398f5f3251a0d47e503c66e935de81230b59b7afb5f41afa8d661cb'
x4='32510ba9aab2a8a4fd06414fb517b5605cc0aa0dc91a8908c2064ba8ad5ea06a029056f47a8ad3306ef5021eafe1ac01a81197847a5c68a1b78769a37bc8f4575432c198ccb4ef63590256e305cd3a9544ee4160ead45aef520489e7da7d835402bca670bda8eb775200b8dabbba246b130f040d8ec6447e2c767f3d30ed81ea2e4c1404e1315a1010e7229be6636aaa'
x5='3f561ba9adb4b6ebec54424ba317b564418fac0dd35f8c08d31a1fe9e24fe56808c213f17c81d9607cee021dafe1e001b21ade877a5e68bea88d61b93ac5ee0d562e8e9582f5ef375f0a4ae20ed86e935de81230b59b73fb4302cd95d770c65b40aaa065f2a5e33a5a0bb5dcaba43722130f042f8ec85b7c2070'
x6='32510bfbacfbb9befd54415da243e1695ecabd58c519cd4bd2061bbde24eb76a19d84aba34d8de287be84d07e7e9a30ee714979c7e1123a8bd9822a33ecaf512472e8e8f8db3f9635c1949e640c621854eba0d79eccf52'
x7='32510bfbacfbb9befd54415da243e1695ecabd58c519cd4bd90f1fa6ea5ba47b01c909ba7696cf606ef40c04afe1ac0aa8148dd066592ded9f8774b529c7ea125d298e8883f5e9305f4b44f915cb2bd05af51373fd9b4af511039fa2d96f83414aaaf261bda2e97b170fb5cce2a53e675c154c0d9681596934777e2275b381ce2e40582afe67650b13e72287'
x8='315c4eeaa8b5f8bd11155ea506b56041c6a00c8a08854dd21a4bbde54ce56801d943ba708b8a3574f40c00'
x9='271946f9bbb2aeadec111841a81abc300ecaa01bd8069d5cc91005e9fe4aad6e04d513e96d99de2569bc5e50eeeca709b50a8a987f4264edb6896fb537d0a716132ddc938fb0f836480e06ed0fcd6e9759f40462f9cf57f4564186a2c1778f1543efa270bda5e933421cbe88a4a52222190f471e9bd15f652b653b7071aec59a2705081'
x10='466d06ece998b7a2fb1d464fed2ced7641ddaa3cc31c9941cf110abbf409ed39598005b3399ccfafb61d0315fca0a314be138a9f32503bedac8067f03adbf3575c3b8edc9ba7f537530541ab0f9f3cd04'

ciphertexts = [target, x1, x2, x3, x4, x5, x6, x7, x8, x9, x10]
cipherlength = len(ciphertexts)

# each byte is 2 characters in hex  
key_length = longest_length(ciphertexts) // 2
keybytes = [0 for i in range(key_length)]

for i in range(cipherlength - 2):
    for j in range(i + 1, cipherlength - 1):
        for k in range(j + 1, cipherlength):
        
            c1hex, c2hex, c3hex = truncated3(ciphertexts[i], ciphertexts[j], ciphertexts[k])
            
            c1 = convert_to_bytes(c1hex)
            c2 = convert_to_bytes(c2hex)
            c3 = convert_to_bytes(c3hex)

            build_key(keybytes, c1, c2, c3)

cbytes = convert_to_bytes(target)
message = decode_raw(cbytes, keybytes)
print(message)

# The secret message is when using stream ciphers never use keys more than once

The secuet message is  Whtn using a stream ciph_r  neveu_·«_d   Tg_Zq Å±ÑÑ¢_
